<a href="https://colab.research.google.com/github/rpastorvargas/reinformentlearning_laura_almon/blob/main/Transformer_over_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download SMAC dataset
!wget https://huggingface.co/datasets/InstaDeepAI/og-marl/resolve/main/core/smac_v1/3m.zip --show-progress

--2024-08-12 17:04:45--  https://huggingface.co/datasets/InstaDeepAI/og-marl/resolve/main/core/smac_v1/3m.zip
Resolving huggingface.co (huggingface.co)... 13.33.30.114, 13.33.30.76, 13.33.30.23, ...
Connecting to huggingface.co (huggingface.co)|13.33.30.114|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/04/13/04136cbae7cbcbff2267631775f640fd149d53371bf076bef89a954916f62466/4cc6dd8462a1811cdf282a9e2308c8c2367f55d6abaf715389abc4e8182f4727?response-content-disposition=inline%3B+filename*%3DUTF-8%27%273m.zip%3B+filename%3D%223m.zip%22%3B&response-content-type=application%2Fzip&Expires=1723741486&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMzc0MTQ4Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzA0LzEzLzA0MTM2Y2JhZTdjYmNiZmYyMjY3NjMxNzc1ZjY0MGZkMTQ5ZDUzMzcxYmYwNzZiZWY4OWE5NTQ5MTZmNjI0NjYvNGNjNmRkODQ2MmExODExY2RmMjgyYTllMjMwOGM4YzIzNjdmNTVkNmFi

In [ ]:
%%capture
!unzip 3m.zip -d vaults

In [ ]:
!ls -la vaults/3m.vlt

total 20
drwxrwxr-x 5 root root 4096 Feb 19 08:01 .
drwxr-xr-x 3 root root 4096 Aug 12 17:05 ..
drwxrwxr-x 3 root root 4096 Feb 19 08:01 Good
drwxrwxr-x 3 root root 4096 Feb 19 08:01 Medium
drwxrwxr-x 3 root root 4096 Feb 19 08:01 Poor


In [ ]:
%%capture
! pip install flashbax~=0.1.2

import jax
import jax.numpy as jnp
import flashbax as fbx
from flashbax.vault import Vault

In [ ]:
vlt = Vault(rel_dir="vaults", vault_name="3m.vlt", vault_uid="Good")

Loading vault found at /content/vaults/3m.vlt/Good


In [ ]:
all_data = vlt.read()

In [ ]:
all_data.keys()

dict_keys(['experience', 'current_index', 'is_full'])

In [ ]:
offline_data = all_data.experience

In [ ]:
offline_data.keys()

dict_keys(['actions', 'infos', 'observations', 'rewards', 'terminals', 'truncations'])

In [ ]:
jax.tree_map(lambda x: x.shape, offline_data)

<ipython-input-10-e323eeb0b801>:1: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  jax.tree_map(lambda x: x.shape, offline_data)


{'actions': (1, 996366, 3),
 'infos': {'legals': (1, 996366, 3, 9), 'state': (1, 996366, 48)},
 'observations': (1, 996366, 3, 30),
 'rewards': (1, 996366, 3),
 'terminals': (1, 996366, 3),
 'truncations': (1, 996366, 3)}

In [ ]:
offline_data['actions'][0][:10]

Array([[4, 2, 4],
       [4, 4, 4],
       [4, 4, 4],
       [4, 4, 4],
       [4, 4, 7],
       [8, 6, 7],
       [8, 4, 7],
       [8, 7, 7],
       [7, 7, 7],
       [7, 7, 7]], dtype=int32)

In [ ]:
import numpy as np
all_data_np = jax.tree_map(lambda x: np.array(x), all_data)

print(all_data_np.experience['actions'][0][:10])

<ipython-input-12-7d6d0b9654c8>:2: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  all_data_np = jax.tree_map(lambda x: np.array(x), all_data)


[[4 2 4]
 [4 4 4]
 [4 4 4]
 [4 4 4]
 [4 4 7]
 [8 6 7]
 [8 4 7]
 [8 7 7]
 [7 7 7]
 [7 7 7]]


In [ ]:
jax.tree_map(lambda x: x.shape, offline_data)

<ipython-input-13-e323eeb0b801>:1: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  jax.tree_map(lambda x: x.shape, offline_data)


{'actions': (1, 996366, 3),
 'infos': {'legals': (1, 996366, 3, 9), 'state': (1, 996366, 48)},
 'observations': (1, 996366, 3, 30),
 'rewards': (1, 996366, 3),
 'terminals': (1, 996366, 3),
 'truncations': (1, 996366, 3)}

In [ ]:
actions = all_data_np.experience['actions'][0]
infos = all_data_np.experience['infos']
infos_legals = infos['legals'][0]
infos_state = infos['state'][0]
observations = all_data_np.experience['observations'][0]
rewards = all_data_np.experience['rewards'][0]
terminals = all_data_np.experience['terminals'][0]
truncations = all_data_np.experience['truncations'][0]

In [ ]:
len(actions), actions[0]

(996366, array([4, 2, 4], dtype=int32))

In [ ]:
# data = []
# episode = {
#     'observations': [],
#     'actions': [],
#     'rewards': []
# }

# for i in range(len(actions)):
#     episode['observations'].append(observations[i])
#     episode['actions'].append(actions[i])
#     episode['rewards'].append(rewards[i])

#     # Check if it is the end of an episode
#     if terminals[i].all() == 1:
#         data.append(episode)
#         episode = {
#             'observations': [],
#             'actions': [],
#             'rewards': []
#         }

In [ ]:
# len(data)

43559

In [ ]:
available_actions = []
for i in range(len(actions)):
    for a in actions[i]:
        if a not in available_actions:
            available_actions.append(a)
available_actions.sort()
print(available_actions)

[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [ ]:
import torch
import torch.nn as nn
from transformers import GPT2Model, GPT2Config

class DecisionTransformer(nn.Module):
    def __init__(self, observation_dim, action_dim, max_length=100):
        super(DecisionTransformer, self).__init__()

        self.observation_dim = observation_dim
        self.action_dim = action_dim
        self.max_length = max_length

        config = GPT2Config(vocab_size=1, n_positions=max_length, n_ctx=max_length, n_embd=256, n_layer=4, n_head=4)
        self.transformer = GPT2Model(config)

        self.embed_observation = nn.Linear(observation_dim, 256)
        self.embed_action = nn.Linear(action_dim, 256)
        self.embed_reward = nn.Linear(3, 256)

        self.action_head = nn.Linear(256, 9)

    def forward(self, observations, actions, rewards):
        # Embed each component
        obs_embeds = self.embed_observation(observations)
        act_embeds = self.embed_action(actions)
        rew_embeds = self.embed_reward(rewards)

        # Concatenate along the sequence dimension
        transformer_input = torch.cat([obs_embeds, act_embeds, rew_embeds], dim=0)

        # Pass through transformer
        transformer_output = self.transformer(inputs_embeds=transformer_input)

        # Predict actions from the transformer output
        predicted_actions = self.action_head(transformer_output.last_hidden_state)

        return predicted_actions

In [ ]:
len(observations), len(observations[0]), len(observations[0][0])

(996366, 3, 30)

In [ ]:
len(actions), actions[0]

(996366, array([4, 2, 4], dtype=int32))

In [ ]:
# n_agents = 3
# data = []

# for i in range(len(actions)):
#     step = {
#         'observations': np.array([]),
#         'actions': [],
#         'rewards': []
#     }
#     for a in range(n_agents):
#         step['observations'] = np.append(step['observations'], observations[i][a])
#         step['actions'].append(actions[i][a])
#         step['rewards'].append(rewards[i][a])
#     data.append(step)

In [ ]:
# n_agents = 3
# data = []

# for i in range(len(actions)):
#     step = {
#         'observations': np.array([]),
#         'actions': [],
#         'rewards': []
#     }
#     step['observations'] = np.append(step['observations'], observations[i][a])
#     step['actions'].append(actions[i][a])
#     step['rewards'].append(rewards[i][a])
#     data.append(step)

In [ ]:
# from transformers import DecisionTransformerModel, DecisionTransformerConfig

# n_agents = 3
# observation_dim = 30 * n_agents  # 30 * 3 = 90
# action_dim = 1 * n_agents  # 1 * 3 = 3

# # Configurar el modelo con los parámetros deseados
# config = DecisionTransformerConfig(
#     state_dim=observation_dim,  # Dimensión de las observaciones
#     act_dim=action_dim,         # Dimensión de las acciones
#     max_length=100,             # Longitud máxima de la secuencia
#     hidden_size=128,            # Tamaño de las capas ocultas
#     num_attention_heads=4,      # Número de cabezas de atención
#     num_layers=2                # Número de capas en el transformador
# )

# model = DecisionTransformerModel(config)

In [ ]:
# ITERATE THROUGH EVERY STEP

import torch.optim as optim

# Initialize the model
n_agents = 3
observation_dim = 30 * n_agents  # 30 * 3 = 90
action_dim = 1 * n_agents  # 1 * 3 = 3
model = DecisionTransformer(observation_dim, action_dim)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()  # Or MSELoss for discrete actions

# Training of the model
def train_model(model, num_epochs=10):
    model.train()

    for epoch in range(num_epochs):
        epoch_loss = 0
        for i in range(len(actions)):
            #observations_tensor = torch.tensor(observations[i], dtype=torch.float32)
            observations_tensor = torch.from_numpy(observations[i]).reshape(1,1,observation_dim).to(dtype=torch.float32)
            #actions_tensor = torch.tensor(actions[i], dtype=torch.float32)
            actions_tensor = torch.from_numpy(actions[i]).reshape(1,1,action_dim).to(dtype=torch.float32)
            #rewards_tensor = torch.tensor(rewards[i], dtype=torch.float32)
            rewards_tensor = torch.from_numpy(rewards[i]).reshape(1,1,n_agents).to(dtype=torch.float32)

            optimizer.zero_grad()

            predicted_actions = model(observations_tensor, actions_tensor, rewards_tensor)

            actions_tensor = actions_tensor[0][0].type(torch.LongTensor)
            predicted_actions = torch.movedim(predicted_actions, 1, 0)[0]

            #predicted_actions = torch.movedim(predicted_actions, (1,2), (0,1))

            loss = criterion(predicted_actions, actions_tensor)
            loss.backward()
            optimizer.step()

            loss_item = loss.item()
            epoch_loss += loss_item

            if i % 10000 == 0:
                print(f"i = {i}, Loss mean = {epoch_loss / (i+1)}")

        print()
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(actions)}")
        print()

train_model(model)

i = 0, Loss mean = 2.579193592071533
i = 10000, Loss mean = 1.046469236371422
i = 20000, Loss mean = 0.9308419639123516
i = 30000, Loss mean = 0.8766793993596816
i = 40000, Loss mean = 0.8452466902257201
i = 50000, Loss mean = 0.8218077601701881
i = 60000, Loss mean = 0.8071355568506285
i = 70000, Loss mean = 0.79727253361365
i = 80000, Loss mean = 0.7859498731437795
i = 90000, Loss mean = 0.7773356637013724
i = 100000, Loss mean = 0.7708500015774488
i = 110000, Loss mean = 0.7644955440236774
i = 120000, Loss mean = 0.7596613824183247
i = 130000, Loss mean = 0.7550380558939842
i = 140000, Loss mean = 0.7508275735462329
i = 150000, Loss mean = 0.7468818002142852
i = 160000, Loss mean = 0.7428667532880868
i = 170000, Loss mean = 0.7398068413058011
i = 180000, Loss mean = 0.7370815539342033
i = 190000, Loss mean = 0.7340212618863475
i = 200000, Loss mean = 0.7313548147219012
i = 210000, Loss mean = 0.7291767650519554
i = 220000, Loss mean = 0.7272456751459798
i = 230000, Loss mean = 0.724

KeyboardInterrupt: 

In [ ]:
torch.save(model, "my-transformer-model")